In [1]:
import re

In [2]:
data = open('data/day5.txt', 'r').read()

In [3]:
example = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [4]:
def part1(data):
    values = data.split('\n\n')
    seeds = [int(seed) for seed in re.findall('\d+', values[0])]
    maps = values[1:]

    for map in maps:
        ranges = [re.findall('\d+', mapping) for mapping in map.split('\n')[1:]]
        for index, seed in enumerate(seeds):
            for (dest_start, source_start, range_len) in ranges:
                # Find where the the seed is in the source range
                if int(source_start) <= seed < int(source_start) + int(range_len):
                    # Calculate the destination value
                    seeds[index] = int(dest_start) + (seed - int(source_start))
    
    return seeds
min(part1(data))

331445006

In [5]:
def part2(data):
    """
    This function operates off the idea that we don't need to convert the full range of values for all seed ranges
    We just need to convert the min and max of any applicable seed ranges and take the minimum of the starting indexes
    """
    values = data.split('\n\n')
    seeds = [int(seed) for seed in re.findall('\d+', values[0])]
    # Turn starts and lengths into seed ranges
    seed_ranges = [(seeds[index - 1], seeds[index - 1] + val) for index, val in enumerate(seeds) if index % 2 != 0]
    maps = values[1:]
    # Add initial seed ranges to mapped range dictionary
    key = ' '.join(re.findall('[a-zA-Z]+', values[0]))
    mapped_ranges = {key: seed_ranges}

    for index, map in enumerate(maps):
        ranges = [re.findall('\d+', mapping) for mapping in map.split('\n')[1:]]
        curr_seed_ranges = mapped_ranges[key]  # retrieve the seed ranges for the current key
        key = ' '.join(re.findall('[a-zA-Z]+', map))
        mapped_ranges[key] = []
        while len(curr_seed_ranges) != 0:
            (seed_range_start, seed_range_end) = curr_seed_ranges.pop()
            added = False
            for (dest_start, source_start, range_len) in ranges:
                # Find where the start of the current seed range is in the source range
                if int(source_start) <= seed_range_start < int(source_start) + int(range_len):
                    new_seed_range_start = int(dest_start) + (seed_range_start - int(source_start))
                    # Handle case where the end of the current seed range is also in the source range
                    if seed_range_end < int(source_start) + int(range_len):
                        new_seed_range_end = int(dest_start) + (seed_range_end - int(source_start))
                        mapped_ranges[key].append((new_seed_range_start, new_seed_range_end))  # add to the next iterations current ranges
                        added = True
                    # Handle case where the end of the current seed range is not in the source range
                    else:
                        new_seed_range_end = int(dest_start) + int(range_len) - 1  # end is the max of the destination range
                        mapped_ranges[key].append((new_seed_range_start, new_seed_range_end))
                        added = True
                        add_seed_range_start = int(source_start) + int(range_len)  # new start is the max of the source range + 1
                        curr_seed_ranges.append((add_seed_range_start, seed_range_end))  # add newly split range to this iterations current ranges
            # Handle case where no appropriate ranges are found
            if not added:
                mapped_ranges[key].append((seed_range_start, seed_range_end))  # add as is to the next iterations current ranges
    
    final_ranges = mapped_ranges[key]  # the final set of mapped ranges are the location ranges
    min_location = min([start for (start, end) in final_ranges])  # the min of the start indexes in the final ranges represents the minimum value
        
    return min_location
part2(data)

6472060